# Exercício

Você nem sempre obterá dados em um formato conveniente, muitas vezes você terá que lidar com dados não numéricos, como nomes de clientes ou códigos postais, nomes de países, etc ...


O Spark possui vários métodos integrados para lidar com essas transformações, verifique todos aqui: http://spark.apache.org/docs/latest/ml-features.html

Vamos ver alguns exemplos de tudo isso!

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('data').getOrCreate()

In [5]:
df = spark.read.csv('fake_customers.csv',inferSchema=True,header=True)

In [6]:
df.show()

+-------+----------+-----+
|   Name|     Phone|Group|
+-------+----------+-----+
|   John|4085552424|    A|
|   Mike|3105552738|    B|
| Cassie|4085552424|    B|
|  Laura|3105552438|    B|
|  Sarah|4085551234|    A|
|  David|3105557463|    C|
|   Zach|4085553987|    C|
|  Kiera|3105552938|    A|
|  Alexa|4085559467|    C|
|Karissa|3105553475|    A|
+-------+----------+-----+



## Recursos de dados

### StringIndexer

Freqüentemente, temos que converter informações de string em informações numéricas como um recurso categórico. Isso é feito facilmente com o Método StringIndexer:

In [7]:
from pyspark.ml.feature import StringIndexer

df = spark.createDataFrame(
    [(0, "a"), (1, "b"), (2, "c"), (3, "a"), (4, "a"), (5, "c")],
    ["user_id", "category"])

indexer = StringIndexer(inputCol="category", outputCol="indice_categorico")
indexed = indexer.fit(df).transform(df)
indexed.show()

+-------+--------+-----------------+
|user_id|category|indice_categorico|
+-------+--------+-----------------+
|      0|       a|              0.0|
|      1|       b|              2.0|
|      2|       c|              1.0|
|      3|       a|              0.0|
|      4|       a|              0.0|
|      5|       c|              1.0|
+-------+--------+-----------------+



### VectorAssembler

VectorAssembler é um transformador que combina uma determinada lista de colunas em uma única coluna de vetor. É útil para combinar recursos brutos e recursos gerados por diferentes transformadores de recursos em um único vetor de recursos, a fim de treinar modelos de ML como regressão logística e árvores de decisão. VectorAssembler aceita os seguintes tipos de coluna de entrada: todos os tipos numéricos, tipo booleano e tipo de vetor. Em cada linha, os valores das colunas de entrada serão concatenados em um vetor na ordem especificada.

Suponha que temos um DataFrame com as colunas id, hour, mobile, userFeatures e clicked:

     id | hour | mobile | userFeatures     | clicked
    ----|------|--------|------------------|---------
     0  | 18   | 1.0    | [0.0, 10.0, 0.5] | 1.0
     
userFeatures é uma coluna vetorial que contém três recursos do usuário. Queremos combinar recursos de hora, dispositivo móvel e usuário em um único vetor de recurso chamado features e usá-lo para prever os cliques ou não. Se definirmos as colunas de entrada de VectorAssembler como hora, móvel e userFeatures e a coluna de saída como recursos, após a transformação, devemos obter o seguinte DataFrame:

     id | hour | mobile | userFeatures     | clicked | features
    ----|------|--------|------------------|---------|-----------------------------
     0  | 18   | 1.0    | [0.0, 10.0, 0.5] | 1.0     | [18.0, 1.0, 0.0, 10.0, 0.5]

In [8]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

dataset = spark.createDataFrame(
    [(0, 18, 1.0, Vectors.dense([0.0, 10.0, 0.5]), 1.0)],
    ["id", "hour", "mobile", "userFeatures", "clicked"])
dataset.show()

+---+----+------+--------------+-------+
| id|hour|mobile|  userFeatures|clicked|
+---+----+------+--------------+-------+
|  0|  18|   1.0|[0.0,10.0,0.5]|    1.0|
+---+----+------+--------------+-------+



In [11]:
assembler = VectorAssembler(
    inputCols=["hour", "mobile", "userFeatures"],
    outputCol="features")

output = assembler.transform(dataset)
print("Assembled columns 'hour', 'mobile', 'userFeatures' to vector column 'features'")
output.select("features", "clicked").show()
output.show()

Assembled columns 'hour', 'mobile', 'userFeatures' to vector column 'features'
+--------------------+-------+
|            features|clicked|
+--------------------+-------+
|[18.0,1.0,0.0,10....|    1.0|
+--------------------+-------+

+---+----+------+--------------+-------+--------------------+
| id|hour|mobile|  userFeatures|clicked|            features|
+---+----+------+--------------+-------+--------------------+
|  0|  18|   1.0|[0.0,10.0,0.5]|    1.0|[18.0,1.0,0.0,10....|
+---+----+------+--------------+-------+--------------------+

